This file allows to analyze results obtained by running experiments_competing_risk.

In [ ]:
import os 
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import sys

sys.path.append('../')
sys.path.append('../DeepSurvivalMachines/')
from nfg import datasets

In [ ]:
# Change this to analyze other datasets result
dataset = 'SYNTHETIC_COMPETING' 
group_selection = 'sex' # For FRAMINGHAM, a more detailed analysis

In [ ]:
path = '../Results/' # Path where the data is saved
x, t, e, cNCriates = datasets.load_dataset(dataset, path = '../', competing = True, normalize = False) # Open the data

In [ ]:
horizons = [0.25, 0.5, 0.75] # Horizons to evaluate the models
times_eval = np.quantile(t[e > 0], horizons)

In [ ]:
from pycox.evaluation import EvalSurv
from sksurv.metrics import concordance_index_ipcw, brier_score, cumulative_dynamic_auc, integrated_brier_score

### Utils: The evaluatino metrics used
def evaluate(survival, e = e, t = t, groups = None, times_eval = []):
    folds = survival.iloc[:, -1].values
    survival = survival.iloc[:, :-1]
    survival.columns = pd.MultiIndex.from_frame(pd.DataFrame(index=survival.columns).reset_index().astype(float))
    
    times = survival.columns.get_level_values(1).unique()
    results = {}

    # If multiple risk, compute cause specific metrics
    for r in survival.columns.get_level_values(0).unique():
        for fold in np.arange(5):
            res = {}
            e_train, t_train = e[folds != fold], t[folds != fold]
            e_test,  t_test  = e[folds == fold], t[folds == fold]
            g_train, g_test = (None, None) if groups is None else (groups[folds != fold], groups[folds == fold])

            et_train = np.array([(e_train[i] == int(r), t_train[i]) for i in range(len(e_train))], # For estimation censoring
                            dtype = [('e', bool), ('t', float)])
            et_test = np.array([(e_test[i] == int(r), t_test[i]) for i in range(len(e_test))], # For measure performance for given outcome
                            dtype = [('e', bool), ('t', float)])
            
            selection = (t_test < t_train.max()) | (e[folds == fold] != int(r))
            
            et_test, g_test = et_test[selection], None if groups is None else g_test[selection]
            survival_train = survival[folds != fold][r]
            survival_fold = survival[folds == fold][r]

            km = EvalSurv(survival_train.T, t_train, e_train == int(r), censor_surv = 'km')
            test_eval = EvalSurv(survival_fold.T, t_test, e_test == int(r), censor_surv = km)

            res['Overall'] = {
                    "CIS": test_eval.concordance_td(), 
                }
            try:
                res['Overall']['BRS'] = test_eval.integrated_brier_score(times.to_numpy())
            except: pass

            
            if len(times_eval) > 0:
                indexes = [np.argmin(np.abs(times - te)) for te in times_eval]
                briers = brier_score(et_train, et_test, survival_fold[selection].iloc[:, indexes], times_eval)[1]
                for te, brier, index in zip(times_eval, briers, indexes):
                    try:
                        res[te] = {
                            "CIS": concordance_index_ipcw(et_train, et_test, 1 - survival_fold[selection].iloc[:, index], te)[0], 
                            "BRS": brier,
                            "ROCS": cumulative_dynamic_auc(et_train, et_test, 1 - survival_fold[selection].iloc[:, index], te)[0][0]}
                    except:
                        pass
                
                    for group in groups.unique() if groups is not None else []:
                        try:
                            res[te]["CIS_{}".format(group)] = concordance_index_ipcw(et_train[g_train == group], et_test[g_test == group], 1 - survival_fold[selection][g_test == group].iloc[:, index], te)[0]
                            res[te]["BRS_{}".format(group)] = brier_score(et_train[g_train == group], et_test[g_test == group], survival_fold[selection][g_test == group].iloc[:, index], [te])[1][0]

                            res[te]["Delta_CIS_{}".format(group)] = res[te]["CIS_{}".format(group)] - concordance_index_ipcw(et_train[g_train != group], et_test[g_test != group], 1 - survival_fold[selection][g_test != group].iloc[:, index], te)[0]
                            res[te]["Delta_BRS_{}".format(group)] = res[te]["BRS_{}".format(group)] - brier_score(et_train[g_train != group], et_test[g_test != group], survival_fold[selection][g_test != group].iloc[:, index], [te])[1][0]
                        
                        except:
                            pass
            results[(r, fold)] = pd.DataFrame.from_dict(res)
    results = pd.concat(results)
    results.index.set_names(['Risk', 'Fold', 'Metric'], inplace = True)

    return results

In [ ]:
# To analyze group performance - We did this only for FRAMINGHAM
if dataset == "FRAMINGHAM":
    if group_selection == 'sex':
        groups = pd.DataFrame(x, columns = cNCriates).SEX.replace({1: 'Male', 2: 'Female'})
        print(groups.value_counts())
        for g in groups.unique():
            print("Group {} - Population {} - Outcome {:.2f}% - Censoring {:.2f}%".format(g, (groups == g).sum(), 100 * (e[groups == g] == 1).mean(),
                                                                                            100 * (e[groups == g] == 0).mean()))
    else:
        groups = pd.DataFrame(x, columns = cNCriates).AGE
        groups = pd.cut(groups, [0, 40, 50, 60, 100], labels=["<40", '40-50', "50-60", "60+"])
        print(groups.value_counts())
        for g in groups.unique().sort_values():
            print("Group {} - Population {} - Outcome {:.2f}% - Censoring {:.2f}%".format(g, (groups == g).sum(), 100 * (e[groups == g] == 2).mean(),
                                                                                        100 * (e[groups == g] == 0).mean()))
else:
    groups = None

In [ ]:
from experiment import Experiment

In [ ]:
# Open file and compute performance
predictions, clusters, results, likelihood, models = {}, {}, {}, {}, {}
for file_name in os.listdir(path):
    if dataset in file_name and '.csv' in file_name: 
        model = file_name       
        model = model[model.rindex('_') + 1: model.index('.')]
        print("Opening :", file_name, ' - ', model)
        
        if 'finegray' in model or 'cox' in model:
            # Reinitialize index
            predictions[model] = pd.read_csv(path + file_name, header = [0], index_col = 0).T.ffill().T
            index = pd.DataFrame([[i, t] for i in ('1', '2') for t in predictions[model].columns[:100]] + [['Use', '']])
            predictions[model].columns = pd.MultiIndex.from_frame(index)
        else:
            predictions[model] = pd.read_csv(path + file_name, header = [0, 1], index_col = 0)

        results[model] = evaluate(predictions[model], groups = groups, times_eval = times_eval)

        model_file = file_name[: file_name.index('.')] + '.pickle'
        try:
            models[model] = Experiment.load(path + model_file)
        except:
            pass

        cluster_file = file_name[: file_name.index('.')] + '_clusters.pickle'
        if os.path.isfile(path + cluster_file):
            clusters[model] = pickle.load(open(path + cluster_file, 'rb'))

# Rename
# TODO: Add your method in the list for nicer display
dict_name = {'nfg': 'NeuralFG', 'nfgcs': 'NeuralFG Non Competing', 'finegray': 'Fine Gray', 'dsm': 'DSM', 'dh': 'DeepHit', 'ds': 'DeSurv', 'coxcs': 'CS Cox'} 

likelihood = pd.DataFrame.from_dict(likelihood, 'index').rename(dict_name)
results = pd.concat(results).rename(dict_name)
results.index.set_names('Model', 0, inplace = True)

In [ ]:
# Compute average performance across fold and models
table = results.groupby(['Model', 'Risk', 'Metric']).apply(lambda x:  pd.Series(["{:.3f} ({:.3f})".format(mean, std) for mean, std in zip(x.mean(), x.std())], index = x.columns))
table = table.unstack(level=-1).stack(level=0).unstack(level=-1).loc[:, ['CIS']]
table = table.reorder_levels(['Risk', 'Model']).sort_index(level = 0, sort_remaining = False)

table

In [ ]:
print(table.droplevel(0, axis = 1).to_latex())

-----

# Split by age

This section is to be used for the FRAMINGHAM analysis.

In [ ]:
assert dataset == "FRAMINGHAM", "All following analysis is specific to FRAMINGHAM"

In [ ]:
selection = ['BRS_Male', 'BRS_Female'] if group_selection == 'sex' else ['BRS_<40', 'BRS_40-50', 'BRS_50-60', 'BRS_60+']

In [ ]:
table = results.groupby(['Model', 'Risk', 'Metric']).apply(lambda x: pd.Series(["{:.3f} ({:.3f})".format(mean, std) for mean, std in zip(x.mean(), x.std())], index = x.columns))
table = table.unstack(level=-1).stack(level=0).loc[['NeuralFG', 'NeuralFG Non Competing'], selection]
table = table.reorder_levels(['Risk', 'Model', None]).sort_index(level = 0, sort_remaining = False)

difference = (results.loc['NeuralFG'] - results.loc['NeuralFG Non Competing']).groupby(['Risk', 'Metric']).apply(lambda x: pd.Series(["{:.3f} ({:.3f})".format(mean, std) for mean, std in zip(x.mean(), x.std())], index = x.columns))
difference = difference.unstack(level=-1).stack(level=0).loc[:, selection]


In [ ]:
table

In [ ]:
table = table.loc[2].T.stack().reorder_levels([None, 'Metric']).sort_index(level = 0, sort_remaining = False)
table['Difference'] = difference.loc[2].stack()
table

In [ ]:
print(table.unstack().T.to_latex())

In [ ]:
print(pd.concat({"Age Group": groups, "Event": pd.Series(e)}, 1).groupby(['Age Group', 'Event']).size().unstack().to_latex())

In [ ]:
pd.concat({"Age Group": groups, "Event": pd.Series(e)}, 1).groupby(['Age Group', 'Event']).size().unstack().rename(columns = {0: 'Censoring', 1: 'Death', 2: 'CVD'})

-------

# Feature importance

Estimate the feature importance of models with and without competing risks to understand how important is to leverage this information. You need to have run the cause specific neural fine gray model (option cause_specific).

In [ ]:
outcome_interest = 2 
iter = 10

In [ ]:
x, t, e, cNCriates = datasets.load_dataset(dataset, competing = True, normalize = True) # Open the data
t =  models['nfg'].__preprocess__(t)

In [ ]:
cNCriates = pd.Series(cNCriates).replace({
    'SEX': 'Sex',
    'CURSMOKE': 'Smoking',
    'DIABETES': 'Diabetes',
    'BPMEDS': 'Anti-hypertensive medication',
    'educ': 'Education',
    'PREVCHD': 'Coronary Heart Disease',
    'PREVAP': 'Angina Pectoris',
    'PREVMI': ' Myocardial Infraction',
    'PREVSTRK': 'Stroke',
    'PREVHYP': 'Hypotension',
    'TOTCHOL': 'Cholesterol',
    'AGE': 'Age',
    'SYSBP': 'Systolic Blood Pressure',
    'DIABP': 'Diastolic Blood Pressure',
    'CIGPDAY': 'Number of cigarettes',
    'HEARTRTE': 'Heart rate',
    'GLUCOSE' : 'Glucose'
}).values

In [ ]:
importance = {'Competing': [], 'Non Competing': []}

for fold in range(5):
    # Competing risk importance
    competing_mean, competing_std = models['nfg'].best_model[fold].feature_importance(x, t, e, iter)
    importance['Competing'].append((pd.Series(competing_mean), pd.Series(competing_std)))

    ncompeting_mean, ncompeting_std = models['nfgcs'].best_model[fold].feature_importance(x, t, e, iter)
    importance['Non Competing'].append((pd.Series(ncompeting_mean), pd.Series(ncompeting_std)))
for model in importance:
    mean, std = pd.concat([impi[0] for impi in importance[model]], axis = 1), pd.concat([impi[1] for impi in importance[model]], axis = 1)
    importance[model] = pd.concat({"Error": std.mean(1), "Mean": mean.mean(1)}, axis = 1) # Wrong error as correlation may imapct

importance = pd.concat(importance, axis = 1)
importance.index = cNCriates

In [ ]:
sort = importance[('Competing', 'Mean')].abs().sort_values().index
importance = importance.loc[sort]

In [ ]:
with sns.axes_style("whitegrid"):
    importance.iloc[-6:,[1, 3]].droplevel(1, axis = 1).plot.barh(xerr = (importance.iloc[-6:,0], importance.iloc[-6:,2]))
    plt.xlim(0, 0.8)
    plt.xlabel('Relative change in NLL')

--------

# Risk sets - Guidelines

Use with the Framingham dataset to explore how this impact the 10-year risk estimation.

In [ ]:
x, t, e, cNCriates = datasets.load_dataset(dataset, competing = True, normalize = True) # Open the data

In [ ]:
ten_year = models['nfg'].__preprocess__(3650) # To adapt if the dataset is not in dayss
labels = ["Low", "Medium", "High"]

In [ ]:
ten_year_survival = {'nfg': pd.Series('', index = predictions['nfg'].index), 'nfgcs': pd.Series('', index = predictions['nfg'].index)}
for fold in range(5):
    index = (predictions['nfg'].Use == fold).iloc[:, 0]

    # Competing risk importance
    for model in ten_year_survival:
        risks = models[model].best_model[fold].predict_risk(x[index], [ten_year], risk = 2).flatten() # Predict CVD risk
        ten_year_survival[model][index] = pd.cut(risks, [0, 0.1, 0.2, 1], labels = labels).to_numpy()

In [ ]:
selection = (groups == '50-60') | (groups == '60+')
analysisGroup = {
    'All': selection,
    'Observed event': (t <= 3650) & (e == 2) & selection,
    'No event': (t > 3650) & selection
}

for group in analysisGroup:
    confusion = pd.DataFrame(confusion_matrix(ten_year_survival['nfg'][analysisGroup[group]], ten_year_survival['nfgcs'][analysisGroup[group]], labels = labels), index = labels, columns = labels) # line represents nfg, columns single nfg
    confusion = pd.concat([confusion, confusion.sum(1).rename('Competing')], 1)
    confusion.loc['Non Competing'] = confusion.sum(0)
    print(group, confusion.to_latex())

---------

# Displaying survival curves

Displays the risk scores estimated by both models

In [ ]:
models_display = ['nfg', 'dh', 'ds', 'dsm', 'finegray', 'coxcs']

In [ ]:
np.random.seed(42)
patient_id = np.random.choice(len(predictions['nfg']), size = 5 )

In [ ]:
fig, axes = plt.subplots(len(patient_id), 2, sharex=True, sharey='row', figsize = (8, len(patient_id) * 2))
fig.supxlabel('Time (in years)')
fig.supylabel('Cumulative Incidence Functions')
plt.xlim(0, 25)

for i, id in enumerate(patient_id):
    for j, risk in enumerate(['1', '2']):
        for model in models_display:
            pred_pat = predictions[model][risk].loc[id].rename(dict_name[model])
            pred_pat.index = pred_pat.index.astype(float) / 365

            pred_pat.plot(legend = False, ax = axes[i, j])
    
        axes[i, j].grid(alpha = 0.3)
    axes[i, 0].set_ylabel(f'Patient {i + 1}')

axes[0, 0].set_title('Death')
axes[0, 1].set_title('CVD')
axes[0, 1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.show()

-------------